In [2]:
import pandas as pd

lng = pd.read_csv('../data/raw/bulgaria/responses-long.csv')

/tmp/ipykernel_74/384077913.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  lng = pd.read_csv('../data/raw/bulgaria/responses-long.csv')


In [ ]:
! pip install --quiet --upgrade vlab-prepro

In [112]:
ll.columns

Index(['parent_surveyid', 'parent_shortcode', 'flowid', 'userid',
       'question_ref', 'question_idx', 'question_text', 'response',
       'timestamp', 'metadata', 'pageid', 'form_lang', 'form_language',
       'form_wave', 'shortcode', 'survey_created', 'survey_name', 'version',
       'final_answer', 'survey_start_time', 'survey_end_time',
       'survey_duration', 'answer_time_min', 'answer_time_median',
       'answer_time_75', 'answer_time_90', 'surveyid', 'translated_response',
       'seed', 'creative'],
      dtype='object')

In [153]:
from vlab_prepro import Preprocessor
from toolz import pipe

def pivot_data(responses):
    p = Preprocessor()

    return pipe(
        responses,
        p.keep_final_answer,
        p.add_metadata(["seed", "creative"]),
        p.drop_users_without("creative"),
        p.drop_duplicated_users(["shortcode"]),
        p.add_duration(),       
        p.pivot("translated_response")
    )

def make_lookup(lng):
    return (lng[lng.response.notna() & lng.translated_response.notna()]
     .groupby(['shortcode', 'question_ref', 'response'])
     .first()
     .reset_index()[['shortcode', 'surveyid', 'question_ref', 'response', 'translated_response']])

def replace_and_pivot(lng):
    lookup = make_lookup(lng)
    ll = lng.drop(columns=["translated_response", "surveyid"]).merge(lookup, on=["shortcode", "question_ref", "response"], how='left')    
    ll['translated_response'] = ll.translated_response.combine_first(ll.response)
    return pivot_data(ll)    

In [121]:
lng = pd.read_csv('../data/raw/serbia/responses-long.csv')

/tmp/ipykernel_74/2545850260.py:1: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  lng = pd.read_csv('../data/raw/serbia/responses-long.csv')


In [122]:
lng.shape

(326113, 29)

In [154]:
lookup = make_lookup(lng)
ll = lng.drop(columns=["translated_response", "surveyid"]).merge(lookup, on=["shortcode", "question_ref", "response"], how='left')    
ll['translated_response'] = ll.translated_response.combine_first(ll.response)


In [ ]:
wide = pivot_data(ll)    

In [ ]:
wide.shape

In [ ]:
wide[wide.duplicated(['userid', 'shortcode'], keep=False)][['userid', 'surveyid', 'shortcode']]

In [143]:
wide.groupby(['userid', 'shortcode']).first().shape

(11332, 102)

In [125]:
dd = pd.read_csv('../data/raw/serbia/responses.csv')

In [133]:
dd.shape

(11267, 105)

In [132]:
dd.groupby(['userid', 'shortcode']).first().shape

(11267, 103)